In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# 많이 다루게 될 데이터 형식
# csv
# json : 8주차때 1~2일정도 다루도록 하겠습니다. 
# txt : 8주차때 1~2일정도 다루도록 하겠습니다.
# xml : 8주차때 1~2일정도 다루도록 하겠습니다.

In [3]:
unzip_data("nlp_getting_started.zip")

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1

In [6]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

id      keyword               location  \
2644  3796  destruction                    NaN   
2227  3185       deluge                    NaN   
5448  7769       police                     UK   
132    191   aftershock                    NaN   
6845  9810       trauma  Montgomery County, MD   

                                                   text  target  
2644  So you have a new weapon that can cause un-ima...       1  
2227  The f$&amp;@ing things I do for #GISHWHES Just...       0  
5448  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...       1  
132   Aftershock back to school kick off was great. ...       0  
6845  in response to trauma Children of Addicts deve...       0

In [7]:
test_df.head()

id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan

In [8]:
# 우리가 tensorflow롤 자연어 데이터를 가지고 할 방향
# Inputs(test column) -> Machine Learning Algirthm -> Outputs (target column)

In [9]:
train_df.target.value_counts()
# 이제 출력값이 2개 라는 것을 알 수 있고, 분류를 한다 binary classification

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(len(train_df)) # 총 학습 데이터양
print(len(test_df)) # 테스트할 데이터양
print(len(train_df) + len(test_df)) # 위에 것 모두 합한 것!

7613
3263
10876


In [11]:
import random
random_index = random.randint(0, len(train_df) - 5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(read disaster)" if target > 0 else "(not real disaster)")
    print(f"text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
text:
@Eric_Tsunami worry about yourself

---

Target: 0 (not real disaster)
text:
#ThingsIhate 
Watching someone you care about head into total destruction and not being able to do anything.

---

Target: 0 (not real disaster)
text:
@sethalphaeus my personal favorites include paramore muse green day royal blood and 5sos

---

Target: 1 (read disaster)
text:
@FlyOpineMonkey You mean Olympic ;-) 
Also its follow-up for Honshu Operation Coronet. 
In all: 1m Allied casualties 30m Japanese dead.

---

Target: 0 (not real disaster)
text:
Fairfax investigating firefighter over Facebook post saying police should be put in Û÷body bagsÛª - The Washington Post http://t.co/jAxHzjCCd4

---



In [12]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size = 0.1,
    random_state = 42
)

In [13]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [15]:
# 우리가 어제까지 전이학습 (transfer learning)을 할 때 숫자 데이터만 가지고 했습니다.
# 왜? 이미지라는 데이터는 기본적으로 숫자를 가지고 있다.

In [16]:
# Tokenization
# 1. word-level
# 2. character-level
# 3. sub-word level

In [17]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
text_vectorizer = TextVectorization(
    max_tokens = None,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None
)

In [19]:
[ len(i.split()) for i in train_sentences ]

[7,
 6,
 20,
 12,
 7,
 17,
 5,
 13,
 25,
 15,
 19,
 10,
 20,
 22,
 9,
 24,
 20,
 6,
 4,
 12,
 8,
 6,
 17,
 17,
 7,
 18,
 19,
 26,
 21,
 10,
 27,
 14,
 5,
 7,
 11,
 7,
 15,
 6,
 11,
 23,
 8,
 17,
 17,
 17,
 3,
 20,
 9,
 9,
 4,
 8,
 19,
 17,
 23,
 20,
 8,
 13,
 19,
 8,
 17,
 9,
 12,
 9,
 25,
 15,
 20,
 8,
 7,
 11,
 12,
 18,
 19,
 10,
 22,
 17,
 11,
 21,
 9,
 16,
 2,
 15,
 24,
 11,
 12,
 20,
 19,
 14,
 16,
 4,
 21,
 9,
 17,
 14,
 11,
 20,
 15,
 12,
 21,
 26,
 12,
 20,
 15,
 15,
 22,
 14,
 11,
 10,
 13,
 11,
 9,
 22,
 22,
 16,
 15,
 5,
 20,
 10,
 15,
 19,
 20,
 10,
 17,
 15,
 12,
 3,
 9,
 3,
 8,
 13,
 8,
 18,
 3,
 15,
 14,
 10,
 22,
 11,
 15,
 21,
 22,
 18,
 8,
 12,
 5,
 20,
 9,
 14,
 7,
 16,
 7,
 18,
 18,
 15,
 7,
 8,
 15,
 28,
 7,
 6,
 11,
 16,
 21,
 11,
 21,
 16,
 18,
 19,
 22,
 7,
 22,
 15,
 19,
 21,
 13,
 21,
 13,
 4,
 23,
 16,
 16,
 16,
 22,
 18,
 22,
 14,
 15,
 18,
 3,
 23,
 11,
 9,
 19,
 24,
 22,
 2,
 9,
 14,
 19,
 14,
 20,
 12,
 11,
 18,
 22,
 18,
 15,
 23,
 22,
 23,
 7,
 7,
 15,


In [20]:
sum( [ len(i.split()) for i in train_sentences ] )  

102087

In [21]:
len(train_sentences)

6851

In [22]:
round( sum( [ len(i.split()) for i in train_sentences ])  / len(train_sentences)) # 각 행의 평균적으로 존재하는 단어의 갯수

15

In [23]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_sequence_length = max_length
)

In [24]:
type(train_sentences)

numpy.ndarray

In [25]:
text_vectorizer.adapt(train_sentences)

In [26]:
sample_sentences = "There's a flood in my street!"
sample_sentences2 = "Oh! My god!"
text_vectorizer([sample_sentences, sample_sentences2])

<tf.Tensor: shape=(2, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [392,  13, 200,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [27]:
# 문장의 단어를 숫자로 잘 만들어주었다!

In [28]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print(text_vectorizer([random_sentence]))

Some curls come out so pretty and some look like I just finished shooting the movie Twister
tf.Tensor(
[[  82    1  220   36   28  766    7   82  173   25    8   29 5640  617
     2]], shape=(1, 15), dtype=int64)


In [29]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [30]:
# 자주 사용되는 token 5개!
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [31]:
# 드물게 사용되는 token 5개
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [32]:
tf.random.set_seed(42)
from tensorflow.keras import layers

In [33]:
embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    embeddings_initializer = "uniform",
    input_length = max_length,
    name = "embedding_1"
)
embedding

In [34]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print(text_vectorizer([random_sentence]))
sample_embed = embedding(text_vectorizer([random_sentence]))
print(sample_embed)

At least 180 dead a million displaced in India floods - India | ReliefWeb http://t.co/0abgFgLH7X
tf.Tensor(
[[  17  506 6359  115    3 1272  463    4  544  207  544 9222    1    0
     0]], shape=(1, 15), dtype=int64)
tf.Tensor(
[[[-0.00224935  0.01572106 -0.04972882 ... -0.00392847  0.030908
   -0.04232835]
  [-0.02652523 -0.02400912  0.02301382 ... -0.0273247   0.00111182
   -0.02807293]
  [-0.04325306  0.04942466  0.00710749 ...  0.01181581 -0.01880198
    0.00973117]
  ...
  [ 0.03977952 -0.03782602 -0.03646283 ...  0.00236253  0.03332629
    0.02803668]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]]], shape=(1, 15, 128), dtype=float32)


In [35]:
sample_embed[0]

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.00224935,  0.01572106, -0.04972882, ..., -0.00392847,
         0.030908  , -0.04232835],
       [-0.02652523, -0.02400912,  0.02301382, ..., -0.0273247 ,
         0.00111182, -0.02807293],
       [-0.04325306,  0.04942466,  0.00710749, ...,  0.01181581,
        -0.01880198,  0.00973117],
       ...,
       [ 0.03977952, -0.03782602, -0.03646283, ...,  0.00236253,
         0.03332629,  0.02803668],
       [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
         0.00912381, -0.00024097],
       [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
         0.00912381, -0.00024097]], dtype=float32)>

In [36]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.00224935,  0.01572106, -0.04972882, -0.04623774,  0.03258872,
       -0.02868998, -0.01195242,  0.0392934 , -0.04054623,  0.0205025 ,
        0.02653402, -0.02134458, -0.04929128, -0.01020234, -0.01770658,
        0.01860027, -0.02741096, -0.01375296, -0.02258855,  0.01718725,
       -0.03837024,  0.0322024 , -0.04695816, -0.03363552,  0.00958283,
        0.03744152, -0.04710681, -0.028232  , -0.00794568, -0.0054406 ,
       -0.01716619,  0.01564279, -0.01001262, -0.0097269 ,  0.00802433,
        0.02680025,  0.02319909, -0.03798115, -0.04496728, -0.02375814,
       -0.02154484,  0.01425952,  0.02938611,  0.01765214,  0.03888377,
        0.04004272,  0.04573157, -0.03730887, -0.04076439,  0.03218165,
       -0.00688662,  0.03806242, -0.00257102, -0.04855287, -0.03286652,
       -0.03970525, -0.04387003,  0.03977134, -0.04712664, -0.01849452,
       -0.03004898, -0.01576607,  0.03455598,  0.02586133, -0.0248903 ,
        0.042259

In [37]:
# Model 0 : Naive Bayes (baseline)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [39]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # tfidf를 통해서 단어를 숫자로 바꾸어주고
    ("clf", MultinomialNB()) # text를 모델링
])

In [40]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [41]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"우리의 기본 baseline 모델은 {baseline_score * 100:.2f}%의 accuracy를 가진다!")

우리의 기본 baseline 모델은 79.27%의 accuracy를 가진다!


In [42]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [43]:
# metrics으로써 acccuracy, precision, recall, f1-score

In [44]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
    model_results = {
        "accuracy" : model_accuracy,
        "precision" : model_precision,
        "recall" : model_recall,
        "f1" : model_f1
    }
    return model_results

In [45]:
baseline_results = calculate_results(
    y_true = val_labels,
    y_pred = baseline_preds
)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [46]:
# Model 1 : 아주 간단한 딥러닝 모델

In [47]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype = "string") # 입력은 1차원 문자열이다!
x = text_vectorizer(inputs) # 문자열을 숫자로!
x = embedding(x) 
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation= "sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")



In [49]:
model_1.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [50]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [51]:
model_1_history = model_1.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [
        create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "simple_dense_model")
    ]
)

TensorBoard log 파일들은 model_logs/simple_dense_model/20211108-192346에 저장했습니다.
Epoch 1/5
215/215 [==============================] - 8s 32ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 4s 16ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [52]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.7874


[0.4766846001148224, 0.787401556968689]

In [53]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.00073165,  0.01504796, -0.03425454, ..., -0.04403543,
         -0.01042278,  0.01876435],
        [ 0.04135866, -0.03945082, -0.03811939, ...,  0.00464736,
          0.03163555,  0.02928304],
        [ 0.00684033,  0.05363132, -0.00241555, ..., -0.07082175,
         -0.047507  ,  0.01448255],
        ...,
        [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
          0.00308807,  0.02215792],
        [ 0.00692342,  0.05942352, -0.01975194, ..., -0.0619906 ,
         -0.01018394,  0.03510419],
        [-0.03723461,  0.06267186, -0.07451147, ..., -0.02367217,
         -0.0864333 ,  0.01742156]], dtype=float32)>]

In [54]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights.shape

(10000, 128)

In [55]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

array([[0.40488204],
       [0.7443312 ],
       [0.99789494],
       [0.10889998],
       [0.11143532],
       [0.93556094],
       [0.9134594 ],
       [0.99253446],
       [0.9715681 ],
       [0.26570344]], dtype=float32)

In [56]:
model_1_pred_probs.shape

(762, 1)

In [57]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [58]:
model_1_results = calculate_results(
    y_true = val_labels,
    y_pred = model_1_preds
)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

In [59]:
np.array( list(model_1_results.values() ) ) > np.array( list(baseline_results.values() ) )

array([False, False, False, False])

In [60]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key} : {value: .2f}, New {key} : {new_model_results[key]:.2f}, Difference: {new_model_results[key] - value:.2f}")

In [61]:
compare_baseline_to_new_results(
    baseline_results = baseline_results,
    new_model_results = model_1_results
)

Baseline accuracy :  79.27, New accuracy : 78.74, Difference: -0.52
Baseline precision :  0.81, New precision : 0.79, Difference: -0.02
Baseline recall :  0.79, New recall : 0.79, Difference: -0.01
Baseline f1 :  0.79, New f1 : 0.78, Difference: -0.00


In [62]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [63]:
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [64]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [69]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights, type(embed_weights), embed_weights.shape

(array([[ 0.00073165,  0.01504796, -0.03425454, ..., -0.04403543,
         -0.01042278,  0.01876435],
        [ 0.04135866, -0.03945082, -0.03811939, ...,  0.00464736,
          0.03163555,  0.02928304],
        [ 0.00684033,  0.05363132, -0.00241555, ..., -0.07082175,
         -0.047507  ,  0.01448255],
        ...,
        [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
          0.00308807,  0.02215792],
        [ 0.00692342,  0.05942352, -0.01975194, ..., -0.0619906 ,
         -0.01018394,  0.03510419],
        [-0.03723461,  0.06267186, -0.07451147, ..., -0.02367217,
         -0.0864333 ,  0.01742156]], dtype=float32),
 numpy.ndarray,
 (10000, 128))

In [ ]:
# Embedding Project Tool을 사용하기 위해
# 1. embeddiing vector (embedding weight)
# 2. embeddiing vector의 meta data (vocabulary)

In [71]:
import io

out_v = io.open("embedding_vectors.tsv", "w", encoding= "utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding= "utf-8")

for num, word in enumerate(words_in_vocab):
    if num == 0:
        continue
    vec = embed_weights[num]
    out_m.write(word + "\n") # 파일에 해당 단어를 쓰기
    out_v.write("\t".join( [ str(x) for x in vec ] ) + "\n") # 해당 단어에 관련있는 word vector를 파일에 기록
out_v.close()
out_m.close()


In [ ]:
# embeddiing vector 파일과 metadata가 준비가 되었습니다.
# Embedding Project로 이동

In [72]:
# Model_2 : LSTM

In [78]:
tf.random.set_seed(42)

model_2_embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    embeddings_initializer = "uniform",
    input_length = max_length,
    name = "embedding_2"
)

inputs = layers.Input(shape = (1, ), dtype = "string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")

(None, 15, 128)
(None, 64)


In [79]:
model_2.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [80]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [82]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(SAVE_DIR, "LSTM")]
)

TensorBoard log 파일들은 model_logs/LSTM/20211108-195422에 저장했습니다.
Epoch 1/5
215/215 [==============================] - 7s 20ms/step - loss: 0.5100 - accuracy: 0.7416 - val_loss: 0.4566 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3176 - accuracy: 0.8717 - val_loss: 0.5138 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2201 - accuracy: 0.9152 - val_loss: 0.5858 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1556 - accuracy: 0.9428 - val_loss: 0.6041 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1076 - accuracy: 0.9594 - val_loss: 0.8746 - val_accuracy: 0.7507


In [83]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1),
 array([[0.00712609],
        [0.7873689 ],
        [0.9996376 ],
        [0.05679151],
        [0.00258219],
        [0.9996238 ],
        [0.9217019 ],
        [0.9997993 ],
        [0.9994955 ],
        [0.664575  ]], dtype=float32))

In [84]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [85]:
model_2_results = calculate_results(
    y_true = val_labels,
    y_pred = model_2_preds
)
model_2_results

{'accuracy': 75.06561679790026,
 'precision': 0.7510077975908164,
 'recall': 0.7506561679790026,
 'f1': 0.7489268622514025}

In [86]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy :  79.27, New accuracy : 75.07, Difference: -4.20
Baseline precision :  0.81, New precision : 0.75, Difference: -0.06
Baseline recall :  0.79, New recall : 0.75, Difference: -0.04
Baseline f1 :  0.79, New f1 : 0.75, Difference: -0.04


In [87]:
# Recurrent Neural Networks (RNN)
# model_2에서 RNN 방식으로 모델을 구성해 보았습니다.
# RNN의 기본 전제는 간단히 이야기해서 다음에 나올 결과를 위해서 과거의 정보를 사용한다.
# 머신러닝도 과거의 데이터에서 패턴을 찾아서 미래를 예측하는 것
# 입력 (X)가 있으면, 과거의 모든 입력을 기반으로 해서 출력 (Y)를 계산

# RNN의 개념은 자연어 텍스트와 같은 sequence를 다룰 때 아주 유용하다!


In [88]:
# 1. Massive earthquake last week, no?
# 2. No massive earthquake last week.
# RNN은 text들의 sequence (숫자로 된 형식)을 계속 확인하고, 여기서 찾은 패턴을 sequence를 기반으로 해서 계속 최신화!

In [ ]:
# RNN은 sequence기반의 문제를 처리하기 위해 사용할 수 있습니다.
# 1. one to one : input 1, output 1 => 이미지 분류
# 2. one to many : input 1, output many => 이미지 설명 (이미지를 입력받고, 이미지의 설명을 출력)
# 3. Many to one : input many, output 1 => 텍스트 분류 
# 4. Many to many : input many, output many => 번역 or STT

In [89]:
# RNN을 사용하는 3가지 유형
# 1. LSTM (Long Short-Term Memory Cells)
# 2. GRU (Gated Recurrent Units)
# 3. Bidirectional RNN

In [90]:
# Model 2 (LSTM을 사용)

# Input (text) -> Tokenize -> Embedding -> Pooling -> Output (label probability) ==> Model 1에서 사용한 방법
# Input (text) -> Tokenize -> Embedding -> LSTM -> Output (label probability) ==> Model 2에서 사용한 방법

In [91]:
# Model 3 : GRU
# GRU : LSTM와 유사하며, parameter 갯수가 더 적다.
# Input (text) -> Tokenize -> Embedding -> GRU -> Output (label probability)

tf.random.set_seed(42)

model_3_embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    embeddings_initializer = "uniform",
    input_length = max_length,
    name = "embedding_3"
)

inputs = layers.Input(shape = (1, ), dtype = "string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
print(x.shape)
x = layers.GRU(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name = "model_3_GRU")


(None, 15, 128)
(None, 64)


In [93]:
model_3.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [94]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [95]:
model_3_history = model_3.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(SAVE_DIR, "GRU")]
)

TensorBoard log 파일들은 model_logs/GRU/20211108-205202에 저장했습니다.
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.5242 - accuracy: 0.7314 - val_loss: 0.4553 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3195 - accuracy: 0.8694 - val_loss: 0.4937 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2197 - accuracy: 0.9181 - val_loss: 0.5607 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1599 - accuracy: 0.9441 - val_loss: 0.6220 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1221 - accuracy: 0.9584 - val_loss: 0.6205 - val_accuracy: 0.7677


In [96]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1),
 array([[0.33325213],
        [0.87741196],
        [0.9980252 ],
        [0.11561713],
        [0.01235959],
        [0.9925637 ],
        [0.6214277 ],
        [0.9981333 ],
        [0.9982377 ],
        [0.5018121 ]], dtype=float32))

In [97]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [98]:
model_3_results = calculate_results(
    y_true = val_labels,
    y_pred = model_3_preds
)
model_3_results

{'accuracy': 76.77165354330708,
 'precision': 0.7675450859410361,
 'recall': 0.7677165354330708,
 'f1': 0.7667932666650168}

In [99]:
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy :  79.27, New accuracy : 76.77, Difference: -2.49
Baseline precision :  0.81, New precision : 0.77, Difference: -0.04
Baseline recall :  0.79, New recall : 0.77, Difference: -0.02
Baseline f1 :  0.79, New f1 : 0.77, Difference: -0.02


In [100]:
# model_4 : Bidirectional RNN
# 표준 RNN의 처리과정을 왼쪽에서 오른쪽으로 방향을 가집니다.
# bidirectional RNN은 왼쪽에서 오른쪽 방향으로 한번 처리하고, 오른쪽에서 왼쪽으로 또한번 처리합니다.
# Bi-RNN는 실제 성능에서 개선을 많이 가지고 있다. 비용 (시간적, 자원적)이 많이 든다!

In [101]:
tf.random.set_seed(42)

model_4_embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    embeddings_initializer = "uniform",
    input_length = max_length,
    name = "embedding_4"
)

inputs = layers.Input(shape = (1, ), dtype = "string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
print(x.shape)
# Bidirectional RNN은 LSTM이 방향을 바꾸어서 2번 실행
x = layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_Bidirectional")

(None, 15, 128)
(None, 128)


In [102]:
model_4.compile(
    loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [103]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")]
)